In [109]:
import pandas as pd
import numpy as np
import random
df=pd.read_csv('tiempos_por_tecla.csv')
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232203 entries, 0 to 232202
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   tiempo_id      232203 non-null  int64
 1   usuario        232203 non-null  int64
 2   practica       232203 non-null  int64
 3   tecla          232203 non-null  int64
 4   up             232203 non-null  int64
 5   down           232203 non-null  int64
 6   anterior       232203 non-null  int64
 7   anterior_down  232203 non-null  int64
 8   modo           232203 non-null  int64
dtypes: int64(9)
memory usage: 15.9 MB


In [110]:
df.head(10)
# sacar promedio de mantener la letra pulsada

,tiempo_id,usuario,practica,tecla,up,down,anterior,anterior_down,modo
0,1,1,1,69,43,0,0,0,1
1,2,1,1,83,237,157,69,0,1
2,3,1,1,84,458,330,83,157,1
3,4,1,1,65,571,426,84,330,1
4,5,1,1,32,650,572,65,426,1
5,6,1,1,69,778,714,32,572,1
6,7,1,1,83,970,858,69,714,1
7,8,1,1,32,1066,986,83,858,1
8,9,1,1,85,1226,1146,32,986,1
9,10,1,1,78,1418,1338,85,1146,1


In [111]:
df['tiempo'] = df.down - df.anterior_down
df['anterior_up'] = df.up.shift(periods = 1)
df['anterior_up'] = np.where(df['down'] == 0, 0, df['anterior_up'])
df['solapa'] = np.where((df.anterior_up - df.down) > 0, 1, 0)
df['tecla'] = np.where(df.tecla == 32, 95, df.tecla)
df['anterior'] = np.where(df.anterior == 32, 95, df.anterior)
df['tecla'] = df['tecla'].apply(chr)
df['anterior'] = df['anterior'].apply(chr)
df['par'] = df.anterior + df.tecla
pt = df.groupby(by=['usuario','practica','modo'])['tiempo'].median()
pt = pt.reset_index()
pt = pt.rename(columns={'tiempo':'mean'})
pa = df.groupby(by=['usuario','practica','modo'])['solapa'].sum()
pa = pa.reset_index()
pl = df.groupby(by=['usuario','practica','modo','par'])['tiempo'].median()
pl = pl.reset_index()
pt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 951 entries, 0 to 950
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   usuario   951 non-null    int64  
 1   practica  951 non-null    int64  
 2   modo      951 non-null    int64  
 3   mean      951 non-null    float64
dtypes: float64(1), int64(3)
memory usage: 29.8 KB


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232203 entries, 0 to 232202
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tiempo_id      232203 non-null  int64  
 1   usuario        232203 non-null  int64  
 2   practica       232203 non-null  int64  
 3   tecla          232203 non-null  object 
 4   up             232203 non-null  int64  
 5   down           232203 non-null  int64  
 6   anterior       232203 non-null  object 
 7   anterior_down  232203 non-null  int64  
 8   modo           232203 non-null  int64  
 9   tiempo         232203 non-null  int64  
 10  anterior_up    232203 non-null  float64
 11  solapa         232203 non-null  int32  
 12  par            232203 non-null  object 
dtypes: float64(1), int32(1), int64(8), object(3)
memory usage: 22.1+ MB


In [113]:
pl = pl.pivot(index=['usuario','practica','modo'], columns='par', values='tiempo')
pl = pl.reset_index()
pl

par,usuario,practica,modo, E,",J",",_",.0,.1,.5,.A,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,1,0.0,NaN,895.5,224.0,320.0,279.5,NaN,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
1,1,1,2,0.0,1059.0,159.0,255.0,271.0,297.0,1009.0,...,208.0,NaN,158.0,383.0,162.0,176.5,226.0,305.0,206.5,167.5
2,1,1,3,136.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2,1,0.0,NaN,200.0,353.0,255.0,264.5,NaN,...,168.0,464.0,144.0,224.0,128.0,192.0,160.0,430.0,678.5,127.5
4,1,2,2,0.0,1029.0,145.0,238.0,399.0,294.5,928.0,...,152.0,NaN,145.0,544.0,125.0,279.5,173.0,336.0,200.0,144.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,95,1,2,0.0,1648.0,1288.0,328.0,152.0,1064.0,792.0,...,436.0,NaN,152.0,192.0,216.0,120.0,160.0,152.0,183.5,144.0
947,96,1,2,0.0,3575.0,728.0,863.0,1419.0,1308.0,1365.0,...,587.5,NaN,261.0,584.0,635.0,952.5,302.0,450.0,258.0,312.5
948,97,1,2,0.0,1221.0,697.0,542.0,398.0,627.0,1121.0,...,438.0,NaN,260.0,710.0,213.0,154.5,1191.0,214.0,387.0,755.0
949,97,2,2,0.0,1260.0,509.0,1254.0,379.0,421.5,1001.0,...,343.5,NaN,149.0,1239.0,189.0,163.0,629.0,294.0,319.0,311.0


In [114]:
ps = df.groupby(by=['usuario','practica','modo'])['tiempo'].std()
ps = ps.reset_index()
ps = ps.rename(columns={'tiempo':'std'})
ps

,usuario,practica,modo,std
0,1,1,1,247.187213
1,1,1,2,232.886798
2,1,1,3,254.744370
3,1,2,1,184.889401
4,1,2,2,200.028626
...,...,...,...,...
946,95,1,2,473.815904
947,96,1,2,518.964295
948,97,1,2,413.758821
949,97,2,2,368.950127


In [115]:
pl

par,usuario,practica,modo, E,",J",",_",.0,.1,.5,.A,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,1,0.0,NaN,895.5,224.0,320.0,279.5,NaN,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
1,1,1,2,0.0,1059.0,159.0,255.0,271.0,297.0,1009.0,...,208.0,NaN,158.0,383.0,162.0,176.5,226.0,305.0,206.5,167.5
2,1,1,3,136.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2,1,0.0,NaN,200.0,353.0,255.0,264.5,NaN,...,168.0,464.0,144.0,224.0,128.0,192.0,160.0,430.0,678.5,127.5
4,1,2,2,0.0,1029.0,145.0,238.0,399.0,294.5,928.0,...,152.0,NaN,145.0,544.0,125.0,279.5,173.0,336.0,200.0,144.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,95,1,2,0.0,1648.0,1288.0,328.0,152.0,1064.0,792.0,...,436.0,NaN,152.0,192.0,216.0,120.0,160.0,152.0,183.5,144.0
947,96,1,2,0.0,3575.0,728.0,863.0,1419.0,1308.0,1365.0,...,587.5,NaN,261.0,584.0,635.0,952.5,302.0,450.0,258.0,312.5
948,97,1,2,0.0,1221.0,697.0,542.0,398.0,627.0,1121.0,...,438.0,NaN,260.0,710.0,213.0,154.5,1191.0,214.0,387.0,755.0
949,97,2,2,0.0,1260.0,509.0,1254.0,379.0,421.5,1001.0,...,343.5,NaN,149.0,1239.0,189.0,163.0,629.0,294.0,319.0,311.0


In [116]:
pp = df.merge(pt, on=['usuario','practica','modo'], how='outer')
pp = pp.merge(ps, on=['usuario','practica','modo'], how='outer')
pp['lec'] = np.where((pp['mean'] + pp['std']) < pp['tiempo'], 1, 0) 
pp

,tiempo_id,usuario,practica,tecla,up,down,anterior,anterior_down,modo,tiempo,anterior_up,solapa,par,mean,std,lec
0,1,1,1,E,43,0, ,0,1,0,0.0,0, E,144.0,247.187213,0
1,2,1,1,S,237,157,E,0,1,157,43.0,0,ES,144.0,247.187213,0
2,3,1,1,T,458,330,S,157,1,173,237.0,0,ST,144.0,247.187213,0
3,4,1,1,A,571,426,T,330,1,96,458.0,1,TA,144.0,247.187213,0
4,5,1,1,_,650,572,A,426,1,146,571.0,0,A_,144.0,247.187213,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232198,232199,79,4,E,43906,43786,I,43735,2,51,43826.0,1,IE,114.5,204.513255,0
232199,232200,79,4,N,43975,43878,E,43786,2,92,43906.0,1,EN,114.5,204.513255,0
232200,232201,79,4,_,44157,44049,N,43878,2,171,43975.0,0,N_,114.5,204.513255,0
232201,232202,79,4,E,44100,44100,_,44049,2,51,44157.0,1,_E,114.5,204.513255,0


In [117]:
pe = pp.groupby(by=['usuario','practica','modo'])['lec'].sum()
pe = pe.reset_index()
pe

,usuario,practica,modo,lec
0,1,1,1,19
1,1,1,2,17
2,1,1,3,7
3,1,2,1,22
4,1,2,2,18
...,...,...,...,...
946,95,1,2,26
947,96,1,2,39
948,97,1,2,45
949,97,2,2,39


In [118]:
pc = pp.groupby(by=['usuario','practica','modo'])['lec'].count()
pc = pc.reset_index()
pc = pc.rename(columns={'lec': 'cant'})
pc

,usuario,practica,modo,cant
0,1,1,1,251
1,1,1,2,248
2,1,1,3,120
3,1,2,1,251
4,1,2,2,248
...,...,...,...,...
946,95,1,2,248
947,96,1,2,248
948,97,1,2,248
949,97,2,2,248


In [119]:
pe = pe.merge(pc, on=['usuario','practica','modo'])
pe

,usuario,practica,modo,lec,cant
0,1,1,1,19,251
1,1,1,2,17,248
2,1,1,3,7,120
3,1,2,1,22,251
4,1,2,2,18,248
...,...,...,...,...,...
946,95,1,2,26,248
947,96,1,2,39,248
948,97,1,2,45,248
949,97,2,2,39,248


In [120]:
pe['mean_error'] = pe['cant'] / pe['lec']
pe

,usuario,practica,modo,lec,cant,mean_error
0,1,1,1,19,251,13.210526
1,1,1,2,17,248,14.588235
2,1,1,3,7,120,17.142857
3,1,2,1,22,251,11.409091
4,1,2,2,18,248,13.777778
...,...,...,...,...,...,...
946,95,1,2,26,248,9.538462
947,96,1,2,39,248,6.358974
948,97,1,2,45,248,5.511111
949,97,2,2,39,248,6.358974


In [121]:
pt

,usuario,practica,modo,mean
0,1,1,1,144.0
1,1,1,2,144.0
2,1,1,3,136.0
3,1,2,1,144.0
4,1,2,2,144.0
...,...,...,...,...
946,95,1,2,144.0
947,96,1,2,241.0
948,97,1,2,173.5
949,97,2,2,164.0


In [122]:
ps

,usuario,practica,modo,std
0,1,1,1,247.187213
1,1,1,2,232.886798
2,1,1,3,254.744370
3,1,2,1,184.889401
4,1,2,2,200.028626
...,...,...,...,...
946,95,1,2,473.815904
947,96,1,2,518.964295
948,97,1,2,413.758821
949,97,2,2,368.950127


In [123]:
pl

par,usuario,practica,modo, E,",J",",_",.0,.1,.5,.A,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,1,0.0,NaN,895.5,224.0,320.0,279.5,NaN,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
1,1,1,2,0.0,1059.0,159.0,255.0,271.0,297.0,1009.0,...,208.0,NaN,158.0,383.0,162.0,176.5,226.0,305.0,206.5,167.5
2,1,1,3,136.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2,1,0.0,NaN,200.0,353.0,255.0,264.5,NaN,...,168.0,464.0,144.0,224.0,128.0,192.0,160.0,430.0,678.5,127.5
4,1,2,2,0.0,1029.0,145.0,238.0,399.0,294.5,928.0,...,152.0,NaN,145.0,544.0,125.0,279.5,173.0,336.0,200.0,144.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,95,1,2,0.0,1648.0,1288.0,328.0,152.0,1064.0,792.0,...,436.0,NaN,152.0,192.0,216.0,120.0,160.0,152.0,183.5,144.0
947,96,1,2,0.0,3575.0,728.0,863.0,1419.0,1308.0,1365.0,...,587.5,NaN,261.0,584.0,635.0,952.5,302.0,450.0,258.0,312.5
948,97,1,2,0.0,1221.0,697.0,542.0,398.0,627.0,1121.0,...,438.0,NaN,260.0,710.0,213.0,154.5,1191.0,214.0,387.0,755.0
949,97,2,2,0.0,1260.0,509.0,1254.0,379.0,421.5,1001.0,...,343.5,NaN,149.0,1239.0,189.0,163.0,629.0,294.0,319.0,311.0


In [124]:
final = pe.merge(pt, on=['usuario','practica','modo'])
final = final.merge(pa, on=['usuario','practica','modo'])
final['prom_solapa'] = final['cant'] / final['solapa'] 
final = final.merge(pl, on=['usuario','practica','modo'])

In [125]:
final = final.drop(columns=['lec','cant','solapa','practica'])
final

,usuario,modo,mean_error,mean,prom_solapa, E,",J",",_",.0,.1,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,13.210526,144.0,4.048387,0.0,NaN,895.5,224.0,320.0,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
1,1,2,14.588235,144.0,4.000000,0.0,1059.0,159.0,255.0,271.0,...,208.0,NaN,158.0,383.0,162.0,176.5,226.0,305.0,206.5,167.5
2,1,3,17.142857,136.0,2.790698,136.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,11.409091,144.0,4.403509,0.0,NaN,200.0,353.0,255.0,...,168.0,464.0,144.0,224.0,128.0,192.0,160.0,430.0,678.5,127.5
4,1,2,13.777778,144.0,3.542857,0.0,1029.0,145.0,238.0,399.0,...,152.0,NaN,145.0,544.0,125.0,279.5,173.0,336.0,200.0,144.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,95,2,9.538462,144.0,2.032787,0.0,1648.0,1288.0,328.0,152.0,...,436.0,NaN,152.0,192.0,216.0,120.0,160.0,152.0,183.5,144.0
947,96,2,6.358974,241.0,19.076923,0.0,3575.0,728.0,863.0,1419.0,...,587.5,NaN,261.0,584.0,635.0,952.5,302.0,450.0,258.0,312.5
948,97,2,5.511111,173.5,3.594203,0.0,1221.0,697.0,542.0,398.0,...,438.0,NaN,260.0,710.0,213.0,154.5,1191.0,214.0,387.0,755.0
949,97,2,6.358974,164.0,3.594203,0.0,1260.0,509.0,1254.0,379.0,...,343.5,NaN,149.0,1239.0,189.0,163.0,629.0,294.0,319.0,311.0


In [126]:
prod = final
prod = prod.where( prod >= 0 , prod['mean'], axis=0)
prod = prod.where( prod != np.inf , 0, axis=0)
prod

,usuario,modo,mean_error,mean,prom_solapa, E,",J",",_",.0,.1,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,13.210526,144.0,4.048387,0.0,144.0,895.5,224.0,320.0,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
1,1,2,14.588235,144.0,4.000000,0.0,1059.0,159.0,255.0,271.0,...,208.0,144.0,158.0,383.0,162.0,176.5,226.0,305.0,206.5,167.5
2,1,3,17.142857,136.0,2.790698,136.0,136.0,136.0,136.0,136.0,...,136.0,136.0,136.0,136.0,136.0,136.0,136.0,136.0,136.0,136.0
3,1,1,11.409091,144.0,4.403509,0.0,144.0,200.0,353.0,255.0,...,168.0,464.0,144.0,224.0,128.0,192.0,160.0,430.0,678.5,127.5
4,1,2,13.777778,144.0,3.542857,0.0,1029.0,145.0,238.0,399.0,...,152.0,144.0,145.0,544.0,125.0,279.5,173.0,336.0,200.0,144.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,95,2,9.538462,144.0,2.032787,0.0,1648.0,1288.0,328.0,152.0,...,436.0,144.0,152.0,192.0,216.0,120.0,160.0,152.0,183.5,144.0
947,96,2,6.358974,241.0,19.076923,0.0,3575.0,728.0,863.0,1419.0,...,587.5,241.0,261.0,584.0,635.0,952.5,302.0,450.0,258.0,312.5
948,97,2,5.511111,173.5,3.594203,0.0,1221.0,697.0,542.0,398.0,...,438.0,173.5,260.0,710.0,213.0,154.5,1191.0,214.0,387.0,755.0
949,97,2,6.358974,164.0,3.594203,0.0,1260.0,509.0,1254.0,379.0,...,343.5,164.0,149.0,1239.0,189.0,163.0,629.0,294.0,319.0,311.0


In [127]:
#prod['usuario'] = np.where(prod['usuario']==1, True, False)
prod

,usuario,modo,mean_error,mean,prom_solapa, E,",J",",_",.0,.1,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,13.210526,144.0,4.048387,0.0,144.0,895.5,224.0,320.0,...,168.0,290.0,145.0,211.0,126.0,175.0,206.0,848.0,184.0,111.5
1,1,2,14.588235,144.0,4.000000,0.0,1059.0,159.0,255.0,271.0,...,208.0,144.0,158.0,383.0,162.0,176.5,226.0,305.0,206.5,167.5
2,1,3,17.142857,136.0,2.790698,136.0,136.0,136.0,136.0,136.0,...,136.0,136.0,136.0,136.0,136.0,136.0,136.0,136.0,136.0,136.0
3,1,1,11.409091,144.0,4.403509,0.0,144.0,200.0,353.0,255.0,...,168.0,464.0,144.0,224.0,128.0,192.0,160.0,430.0,678.5,127.5
4,1,2,13.777778,144.0,3.542857,0.0,1029.0,145.0,238.0,399.0,...,152.0,144.0,145.0,544.0,125.0,279.5,173.0,336.0,200.0,144.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,95,2,9.538462,144.0,2.032787,0.0,1648.0,1288.0,328.0,152.0,...,436.0,144.0,152.0,192.0,216.0,120.0,160.0,152.0,183.5,144.0
947,96,2,6.358974,241.0,19.076923,0.0,3575.0,728.0,863.0,1419.0,...,587.5,241.0,261.0,584.0,635.0,952.5,302.0,450.0,258.0,312.5
948,97,2,5.511111,173.5,3.594203,0.0,1221.0,697.0,542.0,398.0,...,438.0,173.5,260.0,710.0,213.0,154.5,1191.0,214.0,387.0,755.0
949,97,2,6.358974,164.0,3.594203,0.0,1260.0,509.0,1254.0,379.0,...,343.5,164.0,149.0,1239.0,189.0,163.0,629.0,294.0,319.0,311.0


In [171]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model        
from sklearn import neighbors           
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import linear_model
X=prod.drop(axis=1,columns='usuario')  
y=prod['usuario']      
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)
X_val_train,X_val_test,y_val_train,y_val_test=train_test_split(X_train,y_train,test_size=0.20,random_state=123)

k_valores=np.arange(1,51,1)   
k_max=0
MRSEkNN_max=0
for k in k_valores:
    modelokNN = neighbors.KNeighborsClassifier(n_neighbors=k,weights='uniform')
    modelokNN.fit(X_val_train, y_val_train)
    y_val_pred=modelokNN.predict(X_val_test)
    MRSEkNN=accuracy_score(y_val_test, y_val_pred)
    
    if  MRSEkNN>MRSEkNN_max:
        k_max=k
        MRSEkNN_max=MRSEkNN
    
print('Knn con peso uniforme')
print('Valor de k: ', k_max)
print("Mejor MRSE: ", round(MRSEkNN_max,3))

k_valores=np.arange(1,51,1)   
k_max=0
MRSEkNN_max=0
for k in k_valores:
    modelokNN = neighbors.KNeighborsClassifier(n_neighbors=k,weights='distance')
    modelokNN.fit(X_val_train, y_val_train)
    y_val_pred=modelokNN.predict(X_val_test)
    MRSEkNN=accuracy_score(y_val_test, y_val_pred)
    
    if  MRSEkNN>MRSEkNN_max:
        k_max=k
        MRSEkNN_max=MRSEkNN

print('KNn con un peso en distancia')    
print('Valor de k: ', k_max)
print("Mejor MRSE: ", round(MRSEkNN_max,3))



profundidades=np.arange(1,31,1)
p_max=0               
MRSEkNNT_max=0

print("Evaluación modelo Árbol de Decisión en el Validation")

for p in profundidades: 
    modeloT=DecisionTreeClassifier(max_depth=p, random_state=123)        
    modeloT.fit(X_val_train, y_val_train)             
    y_val_pred = modeloT.predict(X_val_test)           
    MRSEkNNT=accuracy_score(y_val_test, y_val_pred)
    
    if  MRSEkNNT>MRSEkNNT_max:
        p_max=p
        MRSEkNNT_max=MRSEkNNT

print('Valor de k: ', p_max)
print("Mejor MRSE: ", round(MRSEkNNT_max,3))

modelo=linear_model.LogisticRegression(max_iter=1000,penalty='none',fit_intercept=True, random_state=123)
modelo.fit(X_val_train,y_val_train)
y_val_pred=modelo.predict(X_val_test)
AC=accuracy_score(y_val_test, y_val_pred)

print("Regrecion: ")
print("Accuracy: ", round(AC,3))



Knn con peso uniforme
Valor de k:  6
Mejor MRSE:  0.612
KNn con un peso en distancia
Valor de k:  6
Mejor MRSE:  0.618
Evaluación modelo Árbol de Decisión en el Validation


In [137]:
res = [random.randrange(0, len(prod), 1) for i in range(20)]
res 
for k in res:
    dt = prod.copy(deep=True)
    dt['usuario'] = np.where( dt['usuario']==k , 1 , 0)
    X=dt.drop(axis=1,columns='usuario')  
    y=dt['usuario']      
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)
    X_val_train,X_val_test,y_val_train,y_val_test=train_test_split(X_train,y_train,test_size=0.20,random_state=123)
    modeloT=DecisionTreeClassifier(max_depth=13, random_state=123)        
    modeloT.fit(X_val_train, y_val_train)             
    y_val_pred = modeloT.predict(X_val_test)           
    MRSEkNNT=accuracy_score(y_val_test, y_val_pred)
    #print(str(p) + "  " + str(k))
    print(MRSEkNNT)
    

1.0
1.0
1.0
1.0
0.9736842105263158
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.993421052631579
1.0
1.0
1.0
1.0
1.0
1.0


In [161]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [169]:
res = prod['usuario'].unique()
res 
top = 0
ton = 0
fn = 0
fp = 0
tn = 0
tp = 0
for k in res:
    dt = prod.copy(deep=True)
    dt['usuario'] = np.where( dt['usuario']==k , 1 , 0)
    X=dt.drop(axis=1,columns='usuario')  
    y=dt['usuario']      
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)
    X_val_train,X_val_test,y_val_train,y_val_test=train_test_split(X_train,y_train,test_size=0.20,random_state=123)
    modeloT=DecisionTreeClassifier(max_depth=13, random_state=123)        
    modeloT.fit(X_val_train, y_val_train)             
    y_val_pred = modeloT.predict(X_val_test)           
    MRSEkNNT=accuracy_score(y_val_test, y_val_pred)
    print(MRSEkNNT)
    y_val_pred = y_val_pred.tolist()
    y_val_test = y_val_test.tolist()
    TP, FP, TN, FN = perf_measure(y_val_pred, y_val_test)
    fn = fn + FN
    fp = fp + FP
    tn = tn + TN
    tp = tp + TP
top = tp + fp
ton = tn + fn
print("total: " + str(t) + " Falso positivo: " + str(fp) + " Falso negativo: " + str(fn) + " Acierto positivo: " + str(tp) + " Acierto negativo: " + str(tn))
print("total: " + str(t) + " Falso positivo: " + str((fp/top)*100) + " Falso negativo: " + str((fn/ton)*100) + " Acierto positivo: " + str((tp/top)*100) + " Acierto negativo: " + str((tn/ton)*100))

0.9605263157894737
0.9671052631578947
0.9144736842105263
0.9605263157894737
0.9868421052631579
0.9539473684210527
1.0
0.9671052631578947
0.9736842105263158
0.9539473684210527
1.0
0.993421052631579
0.9736842105263158
0.9802631578947368
0.9868421052631579
0.993421052631579
0.993421052631579
0.993421052631579
0.9736842105263158
0.993421052631579
1.0
0.9802631578947368
0.9802631578947368
0.993421052631579
0.993421052631579
0.9671052631578947
1.0
0.9868421052631579
0.9868421052631579
0.993421052631579
1.0
1.0
1.0
total: 5016 Falso positivo: 47 Falso negativo: 44 Acierto positivo: 105 Acierto negativo: 4820
total: 5016 Falso positivo: 30.92105263157895 Falso negativo: 0.9046052631578948 Acierto positivo: 69.07894736842105 Acierto negativo: 99.0953947368421


In [146]:
y_val_test = y_val_pred.tolist()
acierto = 0
falson = 0
falsop = 0
for j in range(len(y_val_pred)):
    pred = int(y_val_pred[j])
    test = int(y_val_test[j])
    if pred == test:
        acierto = acierto + 1
        print(pred)
        print(test)
        print("")
    if ((pred == 1)&(test == 0)):
        falsop = falsop + 1
        print(pred)
        print(test)
        print("")
    if ((pred == 0)&(test == 1)):
        falsop = falsop + 1
        print(pred)
        print(test)
        print("")
print("Segun yo: " + str(acierto) + "  Falso negativo: " + str(falson) + "  Falso positivo: " + str(falsop))

152
0
0


In [151]:
for i in range(len(y_val_pred)):
    print(str(y_val_pred[i]) + "/" + str(y_val_test[i]))

0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
